In [ ]:
!nvidia-smi

In [ ]:
!pip install -q \
autodistill \
autodistill-grounded-sam \
autodistill_grounding_dino \
autodistill-yolov8 \
roboflow \
opencv-python \
supervision==0.24.0

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!mkdir {HOME}/images
IMAGE_DIR_PATH = f"{HOME}/images/"

In [ ]:
import supervision as sv

image_paths = sv.list_files_with_extensions(
    directory=IMAGE_DIR_PATH,
    extensions=["png", "jpg"])

print('image count:', len(image_paths))

In [ ]:
SAMPLE_SIZE = 16
SAMPLE_GRID_SIZE = (4,4)
SAMPLE_PLOT_SIZE = (16,16)

In [ ]:
import cv2
import supervision as sv

titles = [
    image_path.stem
    for image_path
    in image_paths[:SAMPLE_SIZE]]
images = [
    cv2.imread(str(image_path))
    for image_path
    in image_paths[:SAMPLE_SIZE]]

sv.plot_images_grid(images=images, titles=titles, grid_size=SAMPLE_GRID_SIZE, size=SAMPLE_PLOT_SIZE)

In [ ]:
from autodistill.detection import CaptionOntology

Ontology=CaptionOntology({
    "NexusAMR": "yellow mobile robot",
})

# Grounding-Dino, for YOLO annotation

In [ ]:
DATASET_DIR_PATH = f"{HOME}/datasetGD"

In [ ]:
from autodistill_grounding_dino import GroundingDINO

model = GroundingDINO(ontology=Ontology)

dataset = model.label(
    input_folder=IMAGE_DIR_PATH,
    extension=".png",
    output_folder=DATASET_DIR_PATH)

Visualization

In [ ]:
ANNOTATIONS_DIRECTORY_PATH = f"{HOME}/datasetGD/train/labels"
IMAGES_DIRECTORY_PATH = f"{HOME}/datasetGD/train/images"
DATA_YAML_PATH = f"{HOME}/datasetGD/data.yaml"

In [ ]:
import supervision as sv

dataset = sv.DetectionDataset.from_yolo(
    images_directory_path=IMAGES_DIRECTORY_PATH,
    annotations_directory_path=ANNOTATIONS_DIRECTORY_PATH,
    data_yaml_path=DATA_YAML_PATH)

len(dataset)

In [ ]:
import supervision as sv
from pathlib import Path


mask_annotator = sv.MaskAnnotator()
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

images = []
image_names = []
for i, (image_path, image, annotation) in enumerate(dataset):
    if i == SAMPLE_SIZE:
        break
    annotated_image = image.copy()
    annotated_image = mask_annotator.annotate(
        scene=annotated_image, detections=annotation)
    annotated_image = box_annotator.annotate(
        scene=annotated_image, detections=annotation)
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=annotation)

    image_names.append(Path(image_path).name)
    images.append(annotated_image)

sv.plot_images_grid(
    images=images,
    titles=image_names,
    grid_size=SAMPLE_GRID_SIZE,
    size=SAMPLE_PLOT_SIZE)

In [ ]:
import shutil

# Path to your folder in Colab
folder_path = DATASET_DIR_PATH

# Zip file path
shutil.make_archive(DATASET_DIR_PATH, 'zip', folder_path)

# Grounded-SAM, for Segmented Mask annotation

In [ ]:
DATASET_DIR_PATH = f"{HOME}/datasetSAM"

In [ ]:
from autodistill_grounded_sam import GroundedSAM

base_model = GroundedSAM(ontology=Ontology)
dataset = base_model.label(
    input_folder=IMAGE_DIR_PATH,
    extension=".png",
    output_folder=DATASET_DIR_PATH)


Visualization

In [ ]:
ANNOTATIONS_DIRECTORY_PATH = f"{HOME}/dataset/train/labels"
IMAGES_DIRECTORY_PATH = f"{HOME}/dataset/train/images"
DATA_YAML_PATH = f"{HOME}/dataset/data.yaml"

In [ ]:
import supervision as sv

dataset = sv.DetectionDataset.from_yolo(
    images_directory_path=IMAGES_DIRECTORY_PATH,
    annotations_directory_path=ANNOTATIONS_DIRECTORY_PATH,
    data_yaml_path=DATA_YAML_PATH)

len(dataset)

In [ ]:
import supervision as sv
from pathlib import Path


mask_annotator = sv.MaskAnnotator()
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

images = []
image_names = []
for i, (image_path, image, annotation) in enumerate(dataset):
    if i == SAMPLE_SIZE:
        break
    annotated_image = image.copy()
    annotated_image = mask_annotator.annotate(
        scene=annotated_image, detections=annotation)
    annotated_image = box_annotator.annotate(
        scene=annotated_image, detections=annotation)
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=annotation)

    image_names.append(Path(image_path).name)
    images.append(annotated_image)

sv.plot_images_grid(
    images=images,
    titles=image_names,
    grid_size=SAMPLE_GRID_SIZE,
    size=SAMPLE_PLOT_SIZE)

In [ ]:
import shutil

# Path to your folder in Colab
folder_path = DATASET_DIR_PATH

# Zip file path
shutil.make_archive(DATASET_DIR_PATH, 'zip', folder_path)